In [11]:
from d2l import torch as d2l
import torch
from torch import  nn

def corr2d(X, K):

    """计算2维的互相关（cross-correlation）
    Y.shape = X-K+1
      W        K        Y
    0 1 2
    3 4 5   * 0 1   = 19 25
    6 7 8     2 3     37 43
    """
    h, w = K.shape
    Y = torch.zeros((X.shape[0]-h + 1, X.shape[1]-w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = d2l.reduce_sum((X[i: i + h,j: j + w] * K))
    return Y

In [12]:
X = torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
K = torch.tensor([[0.0,1.0],[2.0,3.0]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

In [13]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forword(self, x):
        return corr2d(x, self.weight) +self.bias

In [14]:
X = torch.ones(6, 8)
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [15]:
'''用1去检测 边缘从白到黑   -1检测边缘从黑到白'''
K = torch.tensor([[1.0,-1.0]])
Y = corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [16]:
'''得到被检测出来的图片'''
corr2d(d2l.transpose(X),K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [23]:
conv2d = nn.Conv2d(1,1,kernel_size=(1,2), bias=False)
'''(example, channel,height,width)'''
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    #先置零梯度，然后反向传播的到新的梯度
    conv2d.zero_grad()
    l.sum().backward()

    # 更新卷积 w = w - learn_rate*grad
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i + 1) % 2 ==0:
        # print(f'batch{i + 1}, loss {l.sum:.3f}')
        print(f'batch {i + 1}, loss {l.sum():.3f}')

batch 2, loss 7.012
batch 4, loss 2.336
batch 6, loss 0.867
batch 8, loss 0.340
batch 10, loss 0.137


In [24]:
d2l.reshape(conv2d.weight.data,(1, 2))

tensor([[ 0.9543, -1.0302]])

In [ ]:
'''1.如何构造寻找对角线的特征的卷积'''